# Function Approximation with Fully Connected Neural Networks

Exploring how well a plain-vanilla fully connected neural network can approximate arbitrary functions.

In [ ]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import warnings

from math import pi
from matplotlib import pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Use float64 for higher precision
torch.set_default_dtype(torch.float64)

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Target Function

Define the function we want to approximate: `f(x, y) -> z` and evaluate it on a 2D grid to generate the training set.

To experiment with different functions, modify the expression inside `target_function`.

In [ ]:
def target_function(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """
    The function we want to approximate: f(x, y) -> z
    
    Parameters:
        x: Input array of shape (n_samples,)
        y: Input array of shape (n_samples,)
    
    Returns:
        Output array of shape (n_samples,)
    """
    return np.exp(np.sin(pi*x) + y**2)

# ----------------------------------------------------------------

# Generate training data on a 2D grid
X_MIN = -1.0
X_MAX = +1.0
Y_MIN = -1.0
Y_MAX = +1.0
NPOINTS_PER_DIM = 100

# Training set: regular grid
Xs_train = np.linspace(X_MIN, X_MAX, NPOINTS_PER_DIM).astype(np.float64)
Ys_train = np.linspace(Y_MIN, Y_MAX, NPOINTS_PER_DIM).astype(np.float64)
x_grid_train, y_grid_train = np.meshgrid(Xs_train, Ys_train)

x_flat_train = x_grid_train.flatten()
y_flat_train = y_grid_train.flatten()
z_flat_train = target_function(x_flat_train, y_flat_train).astype(np.float64)

# Test set: offset by half the grid spacing (between training points)
step_x = (X_MAX - X_MIN) / (NPOINTS_PER_DIM - 1)
step_y = (Y_MAX - Y_MIN) / (NPOINTS_PER_DIM - 1)
Xs_test = np.linspace(X_MIN + step_x/2, X_MAX - step_x/2, NPOINTS_PER_DIM - 1).astype(np.float64)
Ys_test = np.linspace(Y_MIN + step_y/2, Y_MAX - step_y/2, NPOINTS_PER_DIM - 1).astype(np.float64)
x_grid_test, y_grid_test = np.meshgrid(Xs_test, Ys_test)

x_flat_test = x_grid_test.flatten()
y_flat_test = y_grid_test.flatten()
z_flat_test = target_function(x_flat_test, y_flat_test).astype(np.float64)

# Convert to PyTorch tensors and move to device
X_train = torch.from_numpy(np.stack([x_flat_train, y_flat_train], axis=1)).to(device)
Z_train = torch.from_numpy(z_flat_train).unsqueeze(1).to(device)

X_test = torch.from_numpy(np.stack([x_flat_test, y_flat_test], axis=1)).to(device)
Z_test = torch.from_numpy(z_flat_test).unsqueeze(1).to(device)

print(f"Training set: {X_train.shape[0]} points")
print(f"Test set:     {X_test.shape[0]} points")

## Neural Network

A fully connected (dense) neural network with configurable hidden layers and activation function. The network takes 2 inputs (x, y) and produces 1 output (z).

In [ ]:
class FullyConnectedNetwork(nn.Module):
    """
    A configurable fully connected neural network for function approximation.
    """
    
    def __init__(self, input_dim: int, output_dim: int, hidden_layers: list[int], activation: nn.Module):
        """
        Args:
            input_dim:     Number of input features
            output_dim:    Number of output features
            hidden_layers: List specifying number of neurons in each hidden layer
            activation:    Activation function class to use between layers
        """
        super().__init__()
        
        layers   = []
        prev_dim = input_dim
        
        # Build hidden layers
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(activation())
            prev_dim = hidden_dim
        
        # Add output layer (no activation)
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)

## Training

Supports Adam, L-BFGS, or hybrid (Adam then L-BFGS). Tracks best model and restores it at the end.

In [ ]:
# Configure Training

# set random seed for reproducibility
SEED = 27

# configure the NN
HIDDEN_LAYERS = [24, 24, 24]
ACTIVATION    = nn.Tanh

# configure optimizer
OPTIMIZER_TYPE   = 'adam'  # 'adam', 'lbfgs', or 'hybrid'
NUM_EPOCHS_ADAM  = 1000000
NUM_EPOCHS_LBFGS =    1000
LR_ADAM          =    1e-3
LR_LBFGS         =     0.5

if OPTIMIZER_TYPE == 'adam':
    NUM_EPOCHS  = NUM_EPOCHS_ADAM
    PRINT_EVERY = 10000
elif OPTIMIZER_TYPE == 'lbfgs':
    NUM_EPOCHS = NUM_EPOCHS_LBFGS
    PRINT_EVERY = 100
else:  # hybrid
    NUM_EPOCHS  = NUM_EPOCHS_ADAM + NUM_EPOCHS_LBFGS
    PRINT_EVERY = 1000

# -------------------------------------

# Ensure deterministic behavior
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Create NN
model = FullyConnectedNetwork(2, 1, HIDDEN_LAYERS, ACTIVATION).to(device)

# Log status
n_params = sum(p.numel() for p in model.parameters())
print(f"Network: 2 -> {' -> '.join(map(str, HIDDEN_LAYERS))} -> 1 | {n_params} params | {ACTIVATION.__name__}")
print(f"Random seed: {SEED}")

# Loss function
criterion = nn.MSELoss()

# Training loop
train_losses = []
test_losses = []
lrs = []
best_loss = float('inf')
best_model_state = None
best_epoch = 0
loss_holder = [None]  # For L-BFGS closure
current_optimizer = None
scheduler = None

for epoch in range(NUM_EPOCHS):
    model.train()
    
    # Determine which optimizer to use
    if OPTIMIZER_TYPE == 'adam':
        use_adam = True
    elif OPTIMIZER_TYPE == 'lbfgs':
        use_adam = False
    else:  # hybrid
        use_adam = epoch < NUM_EPOCHS_ADAM
    
    # Initialize or switch optimizer
    if epoch == 0 and (OPTIMIZER_TYPE == 'adam' or OPTIMIZER_TYPE == 'hybrid'):
        current_optimizer = optim.Adam(model.parameters(), lr=LR_ADAM)
        scheduler = ReduceLROnPlateau(current_optimizer, mode='min', factor=0.5, patience=200, min_lr=1e-6)
    elif epoch == 0 and OPTIMIZER_TYPE == 'lbfgs':
        current_optimizer = optim.LBFGS(model.parameters(), lr=LR_LBFGS, max_iter=20, history_size=100)
    elif OPTIMIZER_TYPE == 'hybrid' and epoch == NUM_EPOCHS_ADAM:
        print(f"\n--- Switching to L-BFGS at epoch {epoch} ---\n")
        current_optimizer = optim.LBFGS(model.parameters(), lr=LR_LBFGS, max_iter=20, history_size=100)
        scheduler = None
    
    if use_adam:
        predictions = model(X_train)
        loss = criterion(predictions, Z_train)
        current_optimizer.zero_grad()
        loss.backward()
        current_optimizer.step()
        if scheduler:
            scheduler.step(loss.detach())
        current_train_loss = loss.item()
        lrs.append(current_optimizer.param_groups[0]['lr'])
    else:
        def closure():
            current_optimizer.zero_grad()
            predictions = model(X_train)
            loss = criterion(predictions, Z_train)
            loss.backward()
            loss_holder[0] = loss.item()
            return loss
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            current_optimizer.step(closure)
        current_train_loss = loss_holder[0]
        lrs.append(LR_LBFGS)  # L-BFGS doesn't adapt LR the same way
    
    # Calculate test loss
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test)
        current_test_loss = criterion(test_predictions, Z_test).item()
    
    # Stop if NaN encountered
    if np.isnan(current_train_loss):
        print(f"NaN detected at epoch {epoch + 1}, stopping training.")
        break
    
    train_losses.append(current_train_loss)
    test_losses.append(current_test_loss)
    
    # Track best model (based on training loss)
    if current_train_loss < best_loss:
        best_loss = current_train_loss
        best_model_state = copy.deepcopy(model.state_dict())
        best_epoch = epoch + 1
    
    # Print progress
    print_now = (epoch + 1) % PRINT_EVERY == 0
    if OPTIMIZER_TYPE == 'hybrid' and epoch >= NUM_EPOCHS_ADAM:
        print_now = (epoch - NUM_EPOCHS_ADAM + 1) % 100 == 0  # Print more frequently during L-BFGS
    if print_now:
        phase = "Adam" if use_adam else "L-BFGS"
        print(f"Epoch {epoch + 1:5d}/{NUM_EPOCHS} [{phase:6s}] | Train: {current_train_loss:.2e} | Test: {current_test_loss:.2e} | Best: {best_loss:.2e} | LR: {lrs[-1]:.2e}")

# Restore best model
model.load_state_dict(best_model_state)

print(f"\nO9.19e-04ptimizer: {OPTIMIZER_TYPE}")
print(f"Final train loss: {train_losses[-1]:.2e}")
print(f"Final test loss:  {test_losses[-1]:.2e}")
print(f"Best train loss:  {best_loss:.2e} (epoch {best_epoch})")

In [ ]:
# Plot results

model.eval()
with torch.no_grad():
    z_pred_train = model(X_train).cpu().numpy().squeeze()
    z_pred_test = model(X_test).cpu().numpy().squeeze()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: Training and test loss
ax1 = axes[0]
ax1.semilogy(train_losses, label='Train')
ax1.semilogy(test_losses, label='Test', alpha=0.7)
if OPTIMIZER_TYPE == 'hybrid':
    ax1.axvline(x=NUM_EPOCHS_ADAM, color='r', linestyle='--', label='Switch to L-BFGS')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (log scale)')
ax1.set_title('Training and Test Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Learning rate over time
ax2 = axes[1]
ax2.semilogy(lrs)
if OPTIMIZER_TYPE == 'hybrid':
    ax2.axvline(x=NUM_EPOCHS_ADAM, color='r', linestyle='--', label='Switch to L-BFGS')
    ax2.legend()
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Learning Rate (log scale)')
ax2.set_title('Learning Rate Schedule')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics - Training set
error_train = z_flat_train - z_pred_train
print("Training Set:")
print(f"  Root Mean Squared Error: {np.sqrt(np.mean(error_train**2)):.2e}")
print(f"  Mean Absolute Error:     {np.mean(np.abs(error_train)):.2e}")
print(f"  Max Absolute Error:      {np.max(np.abs(error_train)):.2e}")

# Summary statistics - Test set
error_test = z_flat_test - z_pred_test
print("\nTest Set:")
print(f"  Root Mean Squared Error: {np.sqrt(np.mean(error_test**2)):.2e}")
print(f"  Mean Absolute Error:     {np.mean(np.abs(error_test)):.2e}")
print(f"  Max Absolute Error:      {np.max(np.abs(error_test)):.2e}")

In [ ]:
# RMSE vs Network Size

############################
# function: x * y
# SEED            = 27
# ACTIVATION      = nn.Tanh
# OPTIMIZER_TYPE  = 'adam' 
# LR_ADAM         =  1e-3
# ReduceLROnPlateau(current_optimizer, mode='min', factor=0.5, patience=200, min_lr=1e-6)
############################

# ==========================
# LBFGS

# Two layers
# [6,6] => 67, [8,8] => 105, [16,16] => 337, [32,32] => 1185, [64,64] => 4417, 
# [128, 128] => 17025, [256,256] => 66817, [500,500] => 252501
N_2_LBFGS    = [      67,      105,      337,     1185,     4417,    17025,    66817,   252501]
RMSE_2_LBFGS = [1.38e-03, 1.32e-03, 7.48e-04, 1.18e-03, 9.16e-04, 9.68e-04, 1.43e-03, 6.94e-04]

# Three layers
# [4,4,4] => 57, [8,8,8] => 177, [16,16,16] => 609, [24,24,24] => 1297, [32,32,32] => 2241, 
# [48,48,48] => 4897, [64,64,64] => 8577, [100,100,100] => 20601, 
# [250,250,250] => 126501
N_3_LBFGS    = [      57,      177,     609,     1297,     2241,     4897,     8577,    20601,   126501]
RMSE_3_LBFGS = [1.06e-03, 1.29e-03,9.88e-04, 1.37e-03, 1.49e-03, 9.89e-04, 7.88e-04, 1.25e-03, 1.15e-03]

# Four layers
# [ 5, 5, 5, 5] =>  111, 
# [ 16, 16, 16, 16] =>   881, [24,24,24,24] => 1897,
# [ 32, 32, 32, 32] =>  3297, [48,48,48,48] => 7249
# [ 64, 64, 64, 64] => 12737
# [100,100,100,100] => 30701
# [250,250,250,250] => 189251
N_4_LBFGS    = [     111,      881,     1897,     3297,     7249,    12737,    30701,   189251]
RMSE_4_LBFGS = [1.40e-03, 7.45e-04, 1.09e-03, 8.59e-04, 9.53e-04, 7.71e-04, 1.59e-03, 6.55e-04]

# =========================
# NUM_EPOCHS_ADAM = 100,000

# Two layers
# [6,6] => 67, [8,8] => 105, [16,16] => 337, [32,32] => 1185, [64,64] => 4417, 
# [128, 128] => 17025, [256,256] => 66817, [500,500] => 252501
N_2_ADAM_100k    = [      67,      105,      337,     1185,     4417,    17025,    66817,   252501]
RMSE_2_ADAM_100k = [4.76e-04, 4.95e-04, 2.91e-04, 1.88e-04, 1.30e-04, 9.04e-05, 8.39e-05, 2.33e-04]

# Three layers
# [4,4,4] => 57, [8,8,8] => 177, [16,16,16] => 609, [24,24,24] => 1297, [32,32,32] => 2241
# [48,48,48] => 4897, [64,64,64] => 8577, [100,100,100] => 20601
# [250,250,250] => 126501
N_3_ADAM_100k    = [      57,      177,      609,     1297,     2241,     4897,     8577,    20601,   126501]
RMSE_3_ADAM_100k = [5.70e-04, 4.82e-04, 1.77e-04, 2.36e-04, 3.51e-04, 2.81e-04, 1.78e-04, 1.16e-04, 1.23e-04]

# Four layers
# [ 5, 5, 5, 5] =>  111, 
# [ 16, 16, 16, 16] =>   881, [24,24,24,24] => 1897,
# [ 32, 32, 32, 32] =>  3297, [48,48,48,48] => 7249
# [ 64, 64, 64, 64] => 12737
# [100,100,100,100] => 30701
# [250,250,250,250] => 189251
N_4_ADAM_100k    = [     111,      881,     1897,     3297,     7249,    12737,    30701,   189251]
RMSE_4_ADAM_100k = [9.19e-04, 5.86e-04, 3.90e-04, 3.29e-04, 3.74e-04, 4.15e-04, 3.85e-04, 2.99e-04]

# ===========================
# NUM_EPOCHS_ADAM = 1,000,000

# Two layers
# [6,6] => 67, [8,8] => 105, [16,16] => 337, [32,32] => 1185, [64,64] => 4417, 
# [128, 128] => 17025, [256,256] => 66817, [500,500] => 252501
N_2_ADAM_1M    = [      67,      105,      337,     1185,     4417,    17025,    66817,   252501]
RMSE_2_ADAM_1M = [1.86e-04, 2.00e-04, 5.94e-05, 4.20e-05, 2.79e-05, 2.26e-05, 2.27e-05, 1.44e-05]

# Three layers
# [4,4,4] => 57, [8,8,8] => 177, [16,16,16] => 609, [24,24,24] => 1297, [32,32,32] => 2241
# [48,48,48] => 4897, [64,64,64] => 8577, [100,100,100] => 20601
# [250,250,250] => 126501
N_3_ADAM_1M    = [      57,      177,      609,     1297,     2241,     4897,     8577,    20601,   126501]
RMSE_3_ADAM_1M = [2.45e-04, 1.78e-04, 4.90e-05, 6.78e-05, 4.46e-05, 4.46e-05, 4.21e-05, 3.11e-05, 3.84e-05]

# Four layers
# [ 5, 5, 5, 5] =>  111, 
# [ 16, 16, 16, 16] =>   881, [24,24,24,24] => 1897,
# [ 32, 32, 32, 32] =>  3297, [48,48,48,48] => 7249
# [ 64, 64, 64, 64] => 12737
# [100,100,100,100] => 30701
# [250,250,250,250] => 189251
N_4_ADAM_1M    = [     111,      881,     1897,     3297,     7249,    12737,    30701,   189251]
RMSE_4_ADAM_1M = [4.14e-04, 1.85e-04, 9.01e-05, 6.05e-05, 5.91e-05, 9.47e-05, 5.04e-05, 1.10e-04]

# ============================

# Plot settings
Y_MIN = 0.75e-4  # Set to a value (e.g., 1e-5) to fix the lower y-axis limit, or None for auto
Y_MAX = 1e-2  # Set to a value (e.g., 1e-2) to fix the upper y-axis limit, or None for auto

plt.figure(figsize=(8, 5))
plt.loglog(N_2_LBFGS, RMSE_2_LBFGS, 'o-', label='2 hidden layers')
plt.loglog(N_3_LBFGS, RMSE_3_LBFGS, 's-', label='3 hidden layers')
plt.loglog(N_4_LBFGS, RMSE_4_LBFGS, 's-', label='4 hidden layers')
plt.xlabel('Number of Parameters')
plt.ylabel('RMSE')
plt.title('RMSE vs Network Size: f(x,y) = x * y')
plt.legend()
plt.grid(True, alpha=0.3)
if Y_MIN is not None or Y_MAX is not None:
    plt.ylim(Y_MIN, Y_MAX)
plt.show()

In [ ]:
# RMSE vs Network Size

############################
# function: np.exp(np.sin(pi*x) + y**2)
# SEED            = 27
# ACTIVATION      = nn.Tanh
# OPTIMIZER_TYPE  = 'adam' 
# LR_ADAM         =  1e-3
# ReduceLROnPlateau(current_optimizer, mode='min', factor=0.5, patience=200, min_lr=1e-6)
############################

# ==========================
# LBFGS

# Two layers
# [6,6] => 67, [8,8] => 105, [16,16] => 337, [32,32] => 1185, [64,64] => 4417, 
# [128, 128] => 17025, [256,256] => 66817, [500,500] => 252501
N_2_LBFGS    = [      67,      105,      337,     1185,     4417,    17025,    66817,   252501]
RMSE_2_LBFGS = [3.37e-03, 3.80e-03, 1.45e-03, 1.37e-03, 1.82e-03, 1.61e-03, 1.48e-03, 1.58e-03]

# Three layers
# [4,4,4] => 57, [8,8,8] => 177, [16,16,16] => 609, [24,24,24] => 1297, [32,32,32] => 2241, 
# [48,48,48] => 4897, [64,64,64] => 8577, [100,100,100] => 20601, 
# [250,250,250] => 126501
N_3_LBFGS    = [      57,      177,     609,     1297,    2241,     4897,     8577,    20601,   126501]
RMSE_3_LBFGS = [4.53e-02, 1.93e-03, 1.58e-3, 1.60e-03, 1.24e-3, 1.26e-03, 8.96e-04, 7.70e-04, 1.26e-03]

# Four layers
# [  3,  4,  4,  3] =>    67, [ 5, 5, 5, 5] =>  111, 
# [ 16, 16, 16, 16] =>   881, [24,24,24,24] => 1897,
# [ 32, 32, 32, 32] =>  3297, [48,48,48,48] => 7249
# [ 64, 64, 64, 64] => 12737
# [100,100,100,100] => 30701
# [250,250,250,250] => 189251
N_4_LBFGS    = [      64,      111,      881,     1897,     3297,     7249,     12737,    30701,   189251]
RMSE_4_LBFGS = [3.68e-02, 2.20e-03, 1.81e-03, 2.52e-03, 1.46e-03, 1.13e-03,  1.85e-04, 2.13e-04, 8.85e-04]

# =========================
# NUM_EPOCHS_ADAM = 100,000

# Two layers
# [6,6] => 67, [8,8] => 105, [16,16] => 337, [32,32] => 1185, [64,64] => 4417, 
# [128, 128] => 17025, [256,256] => 66817, [500,500] => 252501
N_2_ADAM_100k    = [      67,      105,      337,     1185,     4417,    17025,    66817,   252501]
RMSE_2_ADAM_100k = [9.42e-03, 2.79e-03, 5.70e-03, 1.29e-03, 9.47e-04, 2.34e-03, 8.04e-04, 5.48e-04]

# Three layers
# [4,4,4] => 57, [8,8,8] => 177, [16,16,16] => 609, [24,24,24] => 1297, [32,32,32] => 2241
# [48,48,48] => 4897, [64,64,64] => 8577, [100,100,100] => 20601
# [250,250,250] => 126501
N_3_ADAM_100k    = [      57,      177,     609,     1297,     2241,     4897,     8577,   20601,   126501]
RMSE_3_ADAM_100k = [1.14e-02, 3.01e-03, 2.05e-3, 7.49e-04, 1.43e-03, 6.63e-04, 5.72e-04, 5.5e-04, 3.42e-04]

# Four layers
# [  3,  4,  4,  3] =>    67, [ 5, 5, 5, 5] =>  111, 
# [ 16, 16, 16, 16] =>   881, [24,24,24,24] => 1897,
# [ 32, 32, 32, 32] =>  3297, [48,48,48,48] => 7249
# [ 64, 64, 64, 64] => 12737
# [100,100,100,100] => 30701
# [250,250,250,250] => 189251
N_4_ADAM_100k    = [      64,      111,      881,     1897,     3297,     7249,    12737,    30701,   189251]
RMSE_4_ADAM_100k = [1.13e-02, 3.51e-03, 1.44e-03, 1.60e-03, 8.67e-04, 7.48e-04, 7.18e-04, 5.07e-04, 2.88e-04]

# ===========================
# NUM_EPOCHS_ADAM = 1,000,000

# Two layers
# [6,6] => 67, [8,8] => 105, [16,16] => 337, [32,32] => 1185, [64,64] => 4417, 
# [128, 128] => 17025, [256,256] => 66817, [500,500] => 252501
N_2_ADAM_1M    = [       67,      105,      337,     1185,    4417,    17025,    66817,    252501]
RMSE_2_ADAM_1M = [ 5.44e-03, 8.97e-04, 9.14e-04, 3.39e-04, 3.07e-4, 2.03e-04, 1.82e-04,  1.52e-04]

# Three layers
# [4,4,4] => 57, [8,8,8] => 177, [16,16,16] => 609, [24,24,24] => 1297, [32,32,32] => 2241
# [48,48,48] => 4897, [64,64,64] => 8577, [100,100,100] => 20601
# [250,250,250] => 126501
N_3_ADAM_1M    = [     57,      177,      609,     1297,     2241,     4897,     8577,    20601,   126501]
RMSE_3_ADAM_1M = [4.07e-3, 1.23e-03, 7.65e-04, 2.06e-04, 2.38e-04, 1.58e-04, 1.54e-04, 1.48e-04, 8.96e-05]

# Four layers
# [  3,  4,  4,  3] =>    67, [ 5, 5, 5, 5] =>  111, 
# [ 16, 16, 16, 16] =>   881, [24,24,24,24] => 1897,
# [ 32, 32, 32, 32] =>  3297, [48,48,48,48] => 7249
# [ 64, 64, 64, 64] => 12737
# [100,100,100,100] => 30701
# [250,250,250,250] => 189251
N_4_ADAM_1M    = [     64,       111,      881,     1897,      3297,     7249,    12737,    30701,   189251]
RMSE_4_ADAM_1M = [4.63e-03, 1.43e-03, 4.33e-04, 2.49e-04,  2.36e-04, 1.85e-04, 2.13e-04, 1.32e-04, 9.41e-05]

# ============================
# NUM_EPOCHS_ADAM = 10,000,000

# Four layers
# [100,100,100,100] =>  30701
# [250,250,250,250] => 189251
#N_4    = [   30701,   189251]
#RMSE_4 = [4.18e-05, 2.37e-05]

# ============================

# Plot settings
Y_MIN = 0.75e-4  # Set to a value (e.g., 1e-5) to fix the lower y-axis limit, or None for auto
Y_MAX = 1e-1  # Set to a value (e.g., 1e-2) to fix the upper y-axis limit, or None for auto

plt.figure(figsize=(8, 5))
plt.loglog(N_2_LBFGS, RMSE_2_LBFGS, 'o-', label='2 hidden layers')
plt.loglog(N_3_LBFGS, RMSE_3_LBFGS, 's-', label='3 hidden layers')
plt.loglog(N_4_LBFGS, RMSE_4_LBFGS, 's-', label='4 hidden layers')
plt.xlabel('Number of Parameters')
plt.ylabel('RMSE')
plt.title('RMSE vs Network Size: f(x,y) = exp(sin(pi*x) + y**2)')
plt.legend()
plt.grid(True, alpha=0.3)
if Y_MIN is not None or Y_MAX is not None:
    plt.ylim(Y_MIN, Y_MAX)
plt.show()